<a href="https://colab.research.google.com/github/tbsoaresvalkms/recipes-ml/blob/master/tensorflow2_0/Transfer%C3%AAncia_de_Aprendizado_e_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Etapa 1: Instalação das dependências

In [0]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 75kB/s 
     |████████████████████████████████| 419kB 49.2MB/s 
     |████████████████████████████████| 3.0MB 32.9MB/s 


In [0]:
!pip install tqdm

### Fazendo o download da base de dados de gatos e cachorros

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2019-09-17 16:12:49--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   106MB/s    in 0.6s    

2019-09-17 16:12:50 (106 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Etapa 2: Pré-processamento

### Importação das bibliotecas

In [0]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0-alpha0'

### Descompactando a base de dados de gatos e cachorros

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

### Configurando os caminhos (paths)

In [0]:
dataset_path_new = "./cats_and_dogs_filtered"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construindo o modelo

### Carregando o modelo pré-treinado (MobileNetV2)

In [0]:
img_shape = (128, 128, 3)

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

In [0]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Congelando o modelo base

In [0]:
base_model.trainable = False

### Definindo o cabeçalho personalizado da rede neural

In [0]:
base_model.output

<tf.Tensor 'out_relu_4/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [0]:
global_average_layer

<tf.Tensor 'global_average_pooling2d_2/Mean:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units = 1, activation = "sigmoid")(global_average_layer)

### Definindo o modelo

In [0]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

### Compilando o modelo

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001),
              loss="binary_crossentropy", metrics = ["accuracy"])

### Criando geradores de dados (Data Generators)

Redimensionando as imagens

    Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: MobileNet (que estamos usando) suporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [0]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Treinando o modelo

In [0]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 74s 5s/step - loss: 0.2396 - accuracy: 0.9020 - val_loss: 0.1202 - val_accuracy: 0.9600
Epoch 2/5
16/16 [==============================] - 12s 728ms/step - loss: 0.0280 - accuracy: 0.9975 - val_loss: 0.1555 - val_accuracy: 0.9490
Epoch 3/5
16/16 [==============================] - 12s 728ms/step - loss: 0.0086 - accuracy: 1.0000 - val_loss: 0.1357 - val_accuracy: 0.9580
Epoch 4/5
16/16 [==============================] - 12s 730ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.1509 - val_accuracy: 0.9610
Epoch 5/5
16/16 [==============================] - 12s 731ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.1695 - val_accuracy: 0.9610


### Avaliação do modelo de transferência de aprendizagem

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
valid_accuracy

0.961

## Fine tuning


Duas questões principais:

- NÃO USE Fine Tuning em toda a rede neural, pois somente em algumas camadas já é suficiente. A ideia é adotar parte específica da rede neural para nosso problema específico
- Inicie o Fine Tuning DEPOIS que você finalizou a transferência de aprendizagem. Se você tentar o Fine Tuning imediatamente, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base 

### Descongelando algumas camadas do topo do modelo base

In [0]:
base_model.trainable = True
len(base_model.layers)

155

In [0]:
fine_tuning_at = 100

In [0]:
for layer in base_model.layers[:fine_tuning_at]:
  layer.trainable = False

### Compilando o modelo para fine tuning

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Fine tuning

In [0]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 39s 2s/step - loss: 5.3824e-04 - accuracy: 1.0000 - val_loss: 0.2278 - val_accuracy: 0.9590
Epoch 2/5
16/16 [==============================] - 8s 508ms/step - loss: 1.1080e-04 - accuracy: 1.0000 - val_loss: 0.2861 - val_accuracy: 0.9490
Epoch 3/5
16/16 [==============================] - 8s 527ms/step - loss: 7.6595e-05 - accuracy: 1.0000 - val_loss: 0.2260 - val_accuracy: 0.9610
Epoch 4/5
16/16 [==============================] - 8s 523ms/step - loss: 3.2351e-05 - accuracy: 1.0000 - val_loss: 0.2823 - val_accuracy: 0.9580
Epoch 5/5
16/16 [==============================] - 8s 523ms/step - loss: 3.3684e-05 - accuracy: 1.0000 - val_loss: 0.1695 - val_accuracy: 0.9660


### Avaliação do modelo com fine tuning

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
valid_accuracy

0.966